In [27]:
from IPython.display import display
import altair as alt

def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)

# Récuperer les données

In [28]:
import duckdb

# Le dataset
csv = '../../data/toflit18_all_flows.csv'

# Lire le csv et l'envoyer comme une relation (table)
toflit_flows = duckdb.sql(f"""
SELECT  customs_region,
        CAST(value AS FLOAT) as value,
        year,
        export_import,
        best_guess_region_prodxpart
        best_guess_national_prodxpart,
        best_guess_national_partner,
        best_guess_national_product,
        best_guess_region_prodxpart,
        best_guess_national_region,
        origin_province,
        partner_grouping,
        partner_simplification
FROM read_csv_auto('{csv}', ALL_VARCHAR=TRUE)
""")

In [29]:
# Appliquer les filtres de base (communes à tous)

base_relation = duckdb.sql(f"""
    FROM toflit_flows
    WHERE customs_region = 'Marseille'
    AND year = '1789'
    AND value IS NOT NULL
    AND export_import = 'Exports'
    AND best_guess_region_prodxpart = '1'
""")

# Filtrer les données en trois catégories

## 1. Exports effectués depuis Marseille vers Marseille

In [30]:
# BUCKET 1 (Marseille) - partner_grouping = 'France' AND partner_simplification = 'Marseille'

marseille_bucket = duckdb.sql(f"""
FROM base_relation
WHERE partner_grouping = 'France'
AND partner_simplification = 'Marseille'
""")

## 2. Exports effectués depuis Marseille vers la France

In [31]:
# BUCKET 2 (France) - partner_grouping = 'France' AND partner_simplification != 'Marseille'

france_bucket = duckdb.sql(f"""
FROM base_relation
WHERE partner_grouping = 'France'
AND partner_simplification != 'Marseille'
""")

## 3. Exports effectués depuis Marseille vers l'étranger

In [32]:
# BUCKET 3 (monde) - partner_grouping != 'France'

monde_bucket = duckdb.sql(f"""
FROM base_relation
WHERE partner_grouping != 'France'
""")

# Des outils d'agrégation et de visualisation

In [33]:
def group_by_province(relation) -> list[dict]:
    aggreg = relation.aggregate("""
        origin_province,
        SUM(value) as total_value
    """)

    values = aggreg.fetchall()
    columns = aggreg.columns

    return [dict(zip(columns, row)) for row in values]

In [34]:
def visualise(aggregations, title):
    
    return alt.Chart().from_dict({
        "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
        "title": title,
        "mark": "bar",
        "data": {
            "values": aggregations
        },
        "encoding": {
            "x": {
                "field": "total_value",
                "type": "quantitative", 
                "title": "value"
            },
            "y": {
                "field": "origin_province",
                "type": "nominal",
                "title": "origin_province",
                "sort": "-x"
            }
        }
    })

# Les résultats

## 1. Marseille vers Marseille

L'origine des produits exportés depuis Marseille et vers Marseille est souvent enregistrée en 1789. La plupart des produits vient du Languedoc, de la Provence, et du Lyonnais.

In [35]:
marseille_bucket_data = group_by_province(marseille_bucket)

In [36]:
chart = visualise(marseille_bucket_data, 'Origine des produits exportés depuis Marseille vers Marseille')
# chart

## 2. Marseille vers la France

L'origine d'aucun produit exporté depuis Marseille vers la France n'est enregistrée.

In [37]:
france_bucket_data = group_by_province(france_bucket)
france_bucket.show()

┌────────────────┬───────────┬─────────┬───────────────┬───┬─────────────────┬──────────────────┬──────────────────────┐
│ customs_region │   value   │  year   │ export_import │ … │ origin_province │ partner_grouping │ partner_simplifica…  │
│    varchar     │   float   │ varchar │    varchar    │   │     varchar     │     varchar      │       varchar        │
├────────────────┼───────────┼─────────┼───────────────┼───┼─────────────────┼──────────────────┼──────────────────────┤
│ Marseille      │ 26325.771 │ 1789    │ Exports       │ … │ NULL            │ France           │ Dunkerque            │
│ Marseille      │   2198.44 │ 1789    │ Exports       │ … │ NULL            │ France           │ Bayonne              │
│ Marseille      │   3015.46 │ 1789    │ Exports       │ … │ NULL            │ France           │ Dunkerque            │
│ Marseille      │     116.0 │ 1789    │ Exports       │ … │ NULL            │ France           │ Corse                │
│ Marseille      │     154.0 │ 1

In [38]:
chart = visualise(france_bucket_data, 'Origine des produits exportés depuis Marseille vers partenaires en France')
# chart

## 3. Marseille vers l'étranger

Une bonne partie des produits exportés depuis Marseille vers l'étranger en 1789 n'a pas d'origine enregristée.

In [39]:
monde_bucket_data = group_by_province(monde_bucket)

In [40]:
chart = visualise(monde_bucket_data, "Origine des produits exportés effectués depuis Marseille vers l'étranger")
# chart

Voir qui sont les partenaires de la marchandise exportée depuis Marseille vers la France 

In [41]:
france_bucket = duckdb.sql(f"""
FROM base_relation
WHERE partner_grouping = 'France'
AND partner_simplification != 'Marseille'
""")

def group_by_partner(relation) -> list[dict]:
    aggreg = relation.aggregate("""
        partner_simplification,
        SUM(value) as total_value,
        COUNT(value) as nb_value,
    """)

    values = aggreg.fetchall()
    columns = aggreg.columns

    return [dict(zip(columns, row)) for row in values]

france_origin_is_null_grouped_by_partner = group_by_partner(france_bucket)

france_origin_is_null_grouped_by_partner

[{'partner_simplification': 'Dunkerque',
  'total_value': 655032.4423675537,
  'nb_value': 43},
 {'partner_simplification': 'Bayonne',
  'total_value': 280171.2106628418,
  'nb_value': 25},
 {'partner_simplification': 'Corse',
  'total_value': 350871.0802078247,
  'nb_value': 48},
 {'partner_simplification': 'Lorient',
  'total_value': 63961.70001220703,
  'nb_value': 4}]